In [11]:
#二产周五的基线需要调低，介于周四和周六之间
#基线日期需手动观察选取

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os
import warnings
import platform

# --- 配置 ---
warnings.filterwarnings('ignore')
# 季节配置 (只用到 anchor)
SEASON_CONFIG = {
    'summer': {'anchor': ('2025-04-01', '2025-05-15')},
    'winter': {'anchor': ('2024-10-11', '2024-11-16')}
}

def load_data(elec_file):
    """只读取电量数据用于计算基线"""
    try:
        df = pd.read_excel(elec_file)
        # 提取日期列
        date_cols = [c for c in df.columns if str(c).isdigit() and len(str(c)) == 8]
        return df, date_cols
    except Exception as e:
        print(f"❌ 读取失败: {e}")
        return None, None

def generate_baseline(elec_file, target_names, output_dir, mode='winter'):
    """生成7天基线并保存"""
    if not os.path.exists(output_dir): os.makedirs(output_dir)
    
    cfg = SEASON_CONFIG[mode]
    print(f"🔄 [步骤1] 正在计算 {mode} 基线 (基准期: {cfg['anchor'][0]} ~ {cfg['anchor'][1]})...")
    
    df_raw, date_cols = load_data(elec_file)
    if df_raw is None: return

    baseline_results = []
    weekdays = {0: '周一', 1: '周二', 2: '周三', 3: '周四', 4: '周五', 5: '周六', 6: '周日'}

    for target in target_names:
        # 1. 提取单行业数据
        industry_row = df_raw[df_raw['行业名称'] == target]
        if industry_row.empty:
            print(f"⚠️ 未找到行业: {target}")
            continue
            
        # 2. 转置为长表
        elec_series = industry_row[date_cols].iloc[0]
        df_elec = pd.DataFrame({
            'date': pd.to_datetime(elec_series.index, format='%Y%m%d'),
            'load': elec_series.values.astype(float)
        })
        
        # 3. 筛选基准期
        mask = (df_elec['date'] >= cfg['anchor'][0]) & (df_elec['date'] <= cfg['anchor'][1])
        df_anchor = df_elec[mask]
        
        # 4. 计算7天基线
        if len(df_anchor) > 0:
            # groupby(dayofweek) 返回 0-6 的索引
            daily_means = df_anchor.groupby(df_anchor['date'].dt.dayofweek)['load'].mean()
            
            # 构建结果行
            row = {'行业名称': target}
            for day_idx in range(7):
                # 如果某天没数据（比如基准期很短且没包含周日），用整体均值填充
                val = daily_means.get(day_idx, df_anchor['load'].mean())
                row[weekdays[day_idx]] = round(val, 2)
            
            baseline_results.append(row)
            print(f"✅ 已计算: {target}")
        else:
            print(f"❌ {target} 基准期无数据")

    # 5. 保存到 Excel
    if baseline_results:
        df_base = pd.DataFrame(baseline_results)
        # 调整列顺序
        cols = ['行业名称', '周一', '周二', '周三', '周四', '周五', '周六', '周日']
        df_base = df_base[cols]
        
        save_path = os.path.join(output_dir, f"{mode}_基线配置表.xlsx")
        df_base.to_excel(save_path, index=False)
        print(f"\n🚀 基线表已生成！请前往修改: {save_path}")
        print("💡 修改完成后，请运行步骤2的脚本。")

if __name__ == "__main__":
    # --- 配置区 ---
    elec_file = r"E:\A智网\月度电力需求预测报告\0_数据\湖北省行业日用电量_26.1.22.xlsx"
    output_dir = r"E:\A智网\月度电力需求预测报告\往期空调电量计算_1.22" 
    
    # 目标行业
    # targets = ["    第二产业"]
    # targets = ["    第三产业"]
    targets = ["B、城乡居民生活用电合计"]
    
    generate_baseline(elec_file, targets, output_dir, mode='winter')

🔄 [步骤1] 正在计算 winter 基线 (基准期: 2024-10-11 ~ 2024-11-16)...
✅ 已计算: B、城乡居民生活用电合计

🚀 基线表已生成！请前往修改: E:\A智网\月度电力需求预测报告\往期空调电量计算_1.22\winter_基线配置表.xlsx
💡 修改完成后，请运行步骤2的脚本。


In [12]:
# --- 1. 全局配置 ---
warnings.filterwarnings('ignore')
system_name = platform.system()
if system_name == 'Windows':
    plt.rcParams['font.sans-serif'] = ['SimHei']
elif system_name == 'Darwin':
    plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']
else:
    plt.rcParams['font.sans-serif'] = ['DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

SEASON_1 = '度冬' # 或 '度夏'

# 季节配置 (这里只需要 focus 核心期，因为基线已经算好了)
SEASON_CONFIG = {
    'summer': {
        'focus':  ('2025-06-15', '2025-09-30'),
        'temp_color': '#2ca02c', 'load_color': '#d62728',
        'label': '制冷电量', 'desc': '骤冷回调'
    },
    'winter': {
        'focus':  ('2025-01-01', '2025-01-22'),
        'temp_color': '#1f77b4', 'load_color': '#ff7f0e',
        'label': '取暖电量', 'desc': '回暖回调'
    }
}

def load_and_merge_data(elec_file, weather_file, target_industry):
    """读取宽表格式的电量和气象数据，并合并"""
    try:
        # 1. 读取电量
        df_elec_raw = pd.read_excel(elec_file)
        industry_row = df_elec_raw[df_elec_raw['行业名称'] == target_industry]
        if industry_row.empty: return None
            
        date_cols = [c for c in df_elec_raw.columns if str(c).isdigit() and len(str(c)) == 8]
        elec_series = industry_row[date_cols].iloc[0]
        df_elec = pd.DataFrame({
            'date': pd.to_datetime(elec_series.index, format='%Y%m%d'),
            'load_MWh': elec_series.values.astype(float)
        })
        
        # 2. 读取气象
        df_weather_raw = pd.read_excel(weather_file)
        if '日期' not in df_weather_raw.columns and 'DATE' not in df_weather_raw.columns:
             df_weather_raw = pd.read_excel(weather_file, header=1)

        date_col = 'DATE' if 'DATE' in df_weather_raw.columns else '日期'
        temp_col = 'AVG' if 'AVG' in df_weather_raw.columns else ('平均温度' if '平均温度' in df_weather_raw.columns else df_weather_raw.columns[3])

        df_weather = df_weather_raw[[date_col, temp_col]].copy()
        df_weather.columns = ['date', 'temp_mean']
        df_weather['date'] = pd.to_datetime(df_weather['date'], errors='coerce')
        df_weather = df_weather.dropna(subset=['date'])
        df_weather['temp_mean'] = pd.to_numeric(df_weather['temp_mean'], errors='coerce')

        # 3. 合并
        df_merged = pd.merge(df_elec, df_weather, on='date', how='left')
        return df_merged.sort_values('date').reset_index(drop=True)

    except Exception as e:
        print(f"❌ 数据读取失败: {e}")
        return None

def auto_detect_events(df, ratio_col='ratio'):
    """自动寻找关键日期"""
    events = {}
    if df.empty: return events
    
    max_idx = df[ratio_col].idxmax()
    events['peak'] = {
        'date': df.loc[max_idx, 'date'],
        'ratio': df.loc[max_idx, ratio_col],
        'load': df.loc[max_idx, 'climate_load'],
        'total_load': df.loc[max_idx, 'load_MWh']
    }
    
    peak_month = events['peak']['date'].month
    df_core = df[df['date'].dt.month == peak_month]
    if not df_core.empty:
        min_idx = df_core[ratio_col].idxmin()
        events['valley'] = {'date': df.loc[min_idx, 'date'], 'ratio': df.loc[min_idx, ratio_col]}
    else:
        events['valley'] = None
    return events

def plot_analysis(df, events, target, mode, cfg, daily_baseline_series, output_folder):
    """绘图函数"""
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True, gridspec_kw={'height_ratios': [2, 1]})
    
    # === 上图 ===
    avg_base = daily_baseline_series.mean()
    ax1.set_title(f"【{target}】{SEASON_1}空调电量\n基线(周均): {avg_base:.0f}", fontsize=14, fontweight='bold')
    
    ax1.stackplot(df['date'], df['fixed_base'], df['climate_load'], 
                  labels=['物理基线', cfg['label']], 
                  colors=['#e0e0e0', cfg['load_color']], alpha=0.6, edgecolor='none')
    
    l2, = ax1.plot(df['date'], df['load_MWh'], color='#000080', linewidth=2, label='总电量')
    l1, = ax1.plot(df['date'], df['fixed_base'], color='#555555', linestyle='--', linewidth=1.5, alpha=0.8)
    
    # 动态Y轴
    y_min, y_max = df['load_MWh'].min(), df['load_MWh'].max()
    y_range = y_max - y_min
    ax1.set_ylim(bottom=max(0, y_min - y_range * 0.15), top=y_max + y_range * 0.15)

    if events.get('peak'):
        pk = events['peak']
        ax1.annotate(f"峰值\n{pk['load']:.0f}", xy=(pk['date'], pk['total_load']), xytext=(0, 20), 
                     textcoords='offset points', arrowprops=dict(arrowstyle='->', color='black'), 
                     ha='center', fontweight='bold', bbox=dict(boxstyle="round,pad=0.3", fc="white", alpha=0.8))

    ax1.set_ylabel('日用电量 (MWh)', fontsize=11)
    ax1.grid(True, linestyle='--', alpha=0.4)
    
    ax1_temp = ax1.twinx()
    l3, = ax1_temp.plot(df['date'], df['temp_mean'], color=cfg['temp_color'], linestyle='-', linewidth=1.5, label='日均气温', alpha=0.8)
    ax1_temp.set_ylabel('气温 (°C)', color=cfg['temp_color'])
    ax1_temp.tick_params(axis='y', labelcolor=cfg['temp_color'])

    import matplotlib.patches as mpatches
    legend_handles = [mpatches.Patch(color='#e0e0e0', label='物理基线'), mpatches.Patch(color=cfg['load_color'], alpha=0.6, label=cfg['label']), l2, l3]
    ax1.legend(handles=legend_handles, loc='upper left', frameon=True, ncol=4)

    # === 下图 ===
    ax2.plot(df['date'], df['ratio'], color=cfg['load_color'], linewidth=2, label='占比')
    ax2.fill_between(df['date'], 0, df['ratio'], color=cfg['load_color'], alpha=0.15)
    
    if events.get('peak'):
        pk = events['peak']
        ax2.annotate(f'Max: {pk["ratio"]:.1f}%', xy=(pk['date'], pk['ratio']), xytext=(0, 10), 
                     textcoords='offset points', color=cfg['load_color'], fontweight='bold', ha='center')

    ax2.set_ylabel(f'{cfg["label"]}占比 (%)', fontsize=11)
    ax2.set_ylim(0, 105)
    ax2.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))
    ax2.xaxis.set_major_locator(mdates.DayLocator(interval=7))
    plt.xticks(rotation=0)
    plt.tight_layout()
    
    save_img = os.path.join(output_folder, f"{mode}_{target}.png")
    plt.savefig(save_img, dpi=300, bbox_inches='tight')
    print(f"📊 图表已保存: {save_img}")
    plt.close()

def run_step2_analysis(elec_file, weather_file, baseline_file, output_folder, mode='summer'):
    """
    【步骤2】读取基线文件 -> 计算 -> 保存Excel -> 绘图
    """
    if not os.path.exists(baseline_file):
        print(f"❌ 找不到基线文件: {baseline_file}\n请先运行步骤1生成基线！")
        return

    print(f"🔄 [步骤2] 读取基线配置: {baseline_file}")
    df_baseline = pd.read_excel(baseline_file)
    cfg = SEASON_CONFIG[mode]
    
    # 星期映射: Excel列名 -> dayofweek index (0-6)
    col_map = {'周一': 0, '周二': 1, '周三': 2, '周四': 3, '周五': 4, '周六': 5, '周日': 6}

    for _, row in df_baseline.iterrows():
        target = row['行业名称']
        
        # 1. 解析该行业的7天基线
        daily_baseline = pd.Series({col_map[k]: row[k] for k in col_map})
        
        # 2. 加载数据
        df = load_and_merge_data(elec_file, weather_file, target)
        if df is None: continue

        # 3. 应用基线
        df['dayofweek'] = df['date'].dt.dayofweek
        df['fixed_base'] = df['dayofweek'].map(daily_baseline)
        
        # 4. 计算气象负荷
        df['climate_load'] = (df['load_MWh'] - df['fixed_base']).clip(lower=0)
        df['ratio'] = np.where(df['load_MWh'] > 0, (df['climate_load'] / df['load_MWh']) * 100, 0)

        # 5. 聚焦核心时段
        focus_mask = (df['date'] >= cfg['focus'][0]) & (df['date'] <= cfg['focus'][1])
        df_focus = df[focus_mask].copy().reset_index(drop=True)
        
        if len(df_focus) == 0:
            print(f"⚠️ [{target}] 核心时段无数据")
            continue

        # 6. 智能侦探
        events = auto_detect_events(df_focus)

        # --- 🔥 [新增] 保存分析结果到 Excel 🔥 ---
        file_name = f"{mode}_{target}_分析结果.xlsx"
        save_path = os.path.join(output_folder, file_name)
        df_focus.to_excel(save_path, index=False)
        print(f"✅ 数据已保存: {save_path}")
        # ---------------------------------------

        # 7. 绘图
        plot_analysis(df_focus, events, target, mode, cfg, daily_baseline, output_folder)

if __name__ == "__main__":
    # --- 配置区 ---
    elec_file = r"E:\A智网\月度电力需求预测报告\0_数据\湖北省行业日用电量_26.1.22.xlsx"
    weather_file = r"E:\A智网\月度电力需求预测报告\0_数据\气象.xlsx"
    output_dir = r"E:\A智网\月度电力需求预测报告\往期空调电量计算_1.22" 
    
    # 这里的基线文件路径要和步骤1生成的一致
    mode = 'winter'
    baseline_file = os.path.join(output_dir, f"{mode}_基线配置表.xlsx")
    
    run_step2_analysis(elec_file, weather_file, baseline_file, output_dir, mode=mode)

🔄 [步骤2] 读取基线配置: E:\A智网\月度电力需求预测报告\往期空调电量计算_1.22\winter_基线配置表.xlsx
✅ 数据已保存: E:\A智网\月度电力需求预测报告\往期空调电量计算_1.22\winter_B、城乡居民生活用电合计_分析结果.xlsx
📊 图表已保存: E:\A智网\月度电力需求预测报告\往期空调电量计算_1.22\winter_B、城乡居民生活用电合计.png
